<img src="files/model_1_outline.png">

In [17]:
## change log
# Epoch to 100 epochs

sequence generator shape (corrected) (b, n, w, h, c)

## Changes: 
### Use camera 2
### Trim to 2 seconds

In [18]:
import os
import glob
import keras

In [19]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
import tensorflow as tf
import datetime

In [20]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [21]:
classes = ['suture_throws', 'hand_ties', 'thread_cuts']
classes.sort()
print(classes)

['hand_ties', 'suture_throws', 'thread_cuts']


In [22]:
# some global params
SIZE = (100, 100) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [23]:
# pattern to get videos and classes
glob_pattern='../data_v2_1_model_1_2/model_data/{classname}/*.avi'

In [24]:
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

In [25]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class hand_ties, validation count: 129, train count: 264
class suture_throws, validation count: 129, train count: 264
class thread_cuts, validation count: 129, train count: 264
Total data: 3 classes for 792 files for train


In [26]:
# getting validation data
valid = train.get_validation_generator()

Total data: 3 classes for 387 files for validation


In [27]:
# import keras_video.utils
# keras_video.utils.show_sample(train)

## BUILD CONV NET

In [28]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
def build_mobilenet(shape=(224, 224, 3), nbout=3):
    model = keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
    output = GlobalMaxPool2D()
    return keras.Sequential([model, output])

## Build GRU

In [29]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [30]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

# this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

## epochs, call backs

In [31]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/20210406-120958


In [32]:
EPOCHS=100
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_1_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/100
99/99 [==============================] - 394s 4s/step - loss: 1.1426 - acc: 0.3788 - val_loss: 0.8429 - val_acc: 0.6250

Epoch 00001: saving model to model2_0_chkp\weights.01-0.84.hdf5
Epoch 2/100
99/99 [==============================] - 27s 278ms/step - loss: 0.8692 - acc: 0.5704 - val_loss: 0.7281 - val_acc: 0.6693

Epoch 00002: saving model to model2_0_chkp\weights.02-0.73.hdf5
Epoch 3/100
99/99 [==============================] - 27s 274ms/step - loss: 0.7347 - acc: 0.6456 - val_loss: 0.6165 - val_acc: 0.7292

Epoch 00003: saving model to model2_0_chkp\weights.03-0.62.hdf5
Epoch 4/100
99/99 [==============================] - 26s 262ms/step - loss: 0.6598 - acc: 0.7089 - val_loss: 0.8693 - val_acc: 0.5990

Epoch 00004: saving model to model2_0_chkp\weights.04-0.87.hdf5
Epoch 5/100
99/99 [==============================] - 26s 261ms/step - loss: 0.6128 - acc: 0.7394 - val_loss: 1.3140 - val_acc: 0.5573

Epoch 00005: saving model to model2_0_chkp\weights.05-1.31.hdf5
Epoch 6


Epoch 00041: saving model to model2_0_chkp\weights.41-0.27.hdf5
Epoch 42/100
99/99 [==============================] - 26s 259ms/step - loss: 0.1217 - acc: 0.9562 - val_loss: 0.2610 - val_acc: 0.9219

Epoch 00042: saving model to model2_0_chkp\weights.42-0.26.hdf5
Epoch 43/100
99/99 [==============================] - 26s 267ms/step - loss: 0.0774 - acc: 0.9714 - val_loss: 0.2679 - val_acc: 0.9193

Epoch 00043: saving model to model2_0_chkp\weights.43-0.27.hdf5
Epoch 44/100
99/99 [==============================] - 26s 266ms/step - loss: 0.0610 - acc: 0.9779 - val_loss: 0.2680 - val_acc: 0.9193

Epoch 00044: saving model to model2_0_chkp\weights.44-0.27.hdf5
Epoch 45/100
99/99 [==============================] - 26s 263ms/step - loss: 0.1223 - acc: 0.9611 - val_loss: 0.2709 - val_acc: 0.9219

Epoch 00045: saving model to model2_0_chkp\weights.45-0.27.hdf5
Epoch 46/100
99/99 [==============================] - 26s 266ms/step - loss: 0.1275 - acc: 0.9680 - val_loss: 0.2748 - val_acc: 0.9141


99/99 [==============================] - 26s 267ms/step - loss: 0.0426 - acc: 0.9899 - val_loss: 0.2939 - val_acc: 0.9193

Epoch 00081: saving model to model2_0_chkp\weights.81-0.29.hdf5
Epoch 82/100
99/99 [==============================] - 26s 264ms/step - loss: 0.0512 - acc: 0.9873 - val_loss: 0.2935 - val_acc: 0.9167

Epoch 00082: saving model to model2_0_chkp\weights.82-0.29.hdf5
Epoch 83/100
99/99 [==============================] - 26s 262ms/step - loss: 0.0694 - acc: 0.9754 - val_loss: 0.2924 - val_acc: 0.9167

Epoch 00083: saving model to model2_0_chkp\weights.83-0.29.hdf5
Epoch 84/100
99/99 [==============================] - 25s 255ms/step - loss: 0.0349 - acc: 0.9883 - val_loss: 0.2909 - val_acc: 0.9167

Epoch 00084: saving model to model2_0_chkp\weights.84-0.29.hdf5
Epoch 85/100
99/99 [==============================] - 26s 266ms/step - loss: 0.0528 - acc: 0.9829 - val_loss: 0.2869 - val_acc: 0.9193

Epoch 00085: saving model to model2_0_chkp\weights.85-0.29.hdf5
Epoch 86/100
